In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import PCA
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import umap.umap_ as umap

class VectorizationEvaluator:
    def __init__(self, documents, stop_words=None):
        """
        Inicialización del evaluador de vectorización
        
        Args:
        - documents: Lista de documentos (texto plano)
        - stop_words: Conjunto de palabras vacías
        """
        self.documents = documents
        self.stop_words = stop_words or set(stopwords.words('spanish'))
    
    def vectorize(self, method='bow', **kwargs):
        """
        Vectorizar documentos usando diferentes métodos
        
        Args:
        - method: Método de vectorización ('bow', 'tfidf', 'word2vec')
        - kwargs: Parámetros adicionales para cada método
        
        Returns:
        - Matriz de vectores
        - Vocabulario
        """
        if method == 'bow':
            vectorizer = CountVectorizer(
                stop_words=list(self.stop_words),
                max_features=kwargs.get('max_features', 1000)
            )
            matrix = vectorizer.fit_transform(self.documents).toarray()
            vocab = vectorizer.get_feature_names_out()
        
        elif method == 'tfidf':
            vectorizer = TfidfVectorizer(
                stop_words=list(self.stop_words),
                max_features=kwargs.get('max_features', 1000)
            )
            matrix = vectorizer.fit_transform(self.documents).toarray()
            vocab = vectorizer.get_feature_names_out()
        
        elif method == 'word2vec':
            # Tokenizar documentos
            tokenized_docs = [
                nltk.word_tokenize(doc.lower()) 
                for doc in self.documents
            ]
            
            # Entrenar modelo Word2Vec
            model = Word2Vec(
                sentences=tokenized_docs, 
                vector_size=kwargs.get('vector_size', 100), 
                window=kwargs.get('window', 5), 
                min_count=kwargs.get('min_count', 1)
            )
            
            # Calcular vector promedio por documento
            matrix = []
            for doc in tokenized_docs:
                doc_vector = np.zeros(model.vector_size)
                count = 0
                for word in doc:
                    if word in model.wv.key_to_index:
                        doc_vector += model.wv[word]
                        count += 1
                
                # Promedio de vectores
                if count > 0:
                    doc_vector /= count
                
                matrix.append(doc_vector)
            
            matrix = np.array(matrix)
            vocab = list(model.wv.key_to_index.keys())
        
        return matrix, vocab
    
    def evaluate_similarity(self, matrix):
        """
        Calcular similaridad de coseno entre documentos
        
        Args:
        - matrix: Matriz de vectores
        
        Returns:
        - Matriz de similaridad
        """
        return cosine_similarity(matrix)
    
    def visualize_embeddings(self, matrix, method='pca'):
        """
        Visualizar embeddings en 2D
        
        Args:
        - matrix: Matriz de vectores
        - method: Método de reducción de dimensionalidad
        
        Returns:
        - Coordenadas 2D
        """
        if method == 'pca':
            reducer = PCA(n_components=2)
        elif method == 'umap':
            reducer = umap.UMAP(n_components=2)
        
        # Reducir dimensionalidad
        coords = reducer.fit_transform(matrix)
        return coords
    
    def analyze_vocabulary(self, vocab):
        """
        Analizar características del vocabulario
        
        Args:
        - vocab: Vocabulario de términos
        
        Returns:
        - Diccionario con estadísticas del vocabulario
        """
        return {
            'total_terms': len(vocab),
            'term_lengths': [len(term) for term in vocab],
            'avg_term_length': np.mean([len(term) for term in vocab]),
            'unique_terms': len(set(vocab))
        }
    
    def comprehensive_evaluation(self, methods=['bow', 'tfidf', 'word2vec']):
        """
        Evaluación completa de métodos de vectorización
        
        Args:
        - methods: Lista de métodos a evaluar
        
        Returns:
        - Resultados de evaluación
        """
        results = {}
        
        for method in methods:
            # Vectorizar
            matrix, vocab = self.vectorize(method)
            
            # Similaridad
            similarity_matrix = self.evaluate_similarity(matrix)
            
            # Visualización
            coords = self.visualize_embeddings(matrix, 'umap')
            
            # Análisis de vocabulario
            vocab_stats = self.analyze_vocabulary(vocab)
            
            # Guardar resultados
            results[method] = {
                'matrix': matrix,
                'vocab': vocab,
                'similarity_matrix': similarity_matrix,
                'embedding_coords': coords,
                'vocab_stats': vocab_stats
            }
        
        return results
    
    def plot_results(self, results):
        """
        Graficar resultados de evaluación
        
        Args:
        - results: Resultados de evaluación
        """
        plt.figure(figsize=(15, 5))
        
        # Gráfico de dispersión de embeddings
        plt.subplot(131)
        for method, data in results.items():
            plt.scatter(
                data['embedding_coords'][:, 0], 
                data['embedding_coords'][:, 1], 
                label=method
            )
        plt.title('Embeddings de Documentos')
        plt.legend()
        
        # Heatmap de similaridad
        plt.subplot(132)
        sns.heatmap(
            results['tfidf']['similarity_matrix'], 
            cmap='YlGnBu', 
            annot=True, 
            fmt='.2f'
        )
        plt.title('Matriz de Similaridad')
        
        # Estadísticas de vocabulario
        plt.subplot(133)
        vocab_stats = [
            results[method]['vocab_stats']['total_terms'] 
            for method in results.keys()
        ]
        plt.bar(results.keys(), vocab_stats)
        plt.title('Términos por Método')
        plt.ylabel('Número de Términos')
        
        plt.tight_layout()
        plt.show()

# Ejemplo de uso
documentos = [
    "El análisis de datos es fundamental en ciencia",
    "La inteligencia artificial transforma la tecnología",
    "Los modelos de machine learning son complejos",
    "La estadística ayuda a comprender los datos",
    "La ciencia de datos combina múltiples disciplinas"
]

# Inicializar evaluador
evaluador = VectorizationEvaluator(documentos)

# Realizar evaluación completa
resultados = evaluador.comprehensive_evaluation()

# Imprimir resultados detallados
for metodo, datos in resultados.items():
    print(f"\n--- Resultados para {metodo.upper()} ---")
    print("Estadísticas de Vocabulario:")
    for stat, valor in datos['vocab_stats'].items():
        print(f"{stat}: {valor}")
    
    print("\nMatriz de Vectores (primeras 2 filas):")
    print(datos['matrix'][:2])

# Graficar resultados
evaluador.plot_results(resultados)

# Análisis comparativo detallado
def analisis_comparativo(resultados):
    print("\n=== ANÁLISIS COMPARATIVO DE TÉCNICAS DE VECTORIZACIÓN ===")
    
    # Comparación de dimensionalidad
    print("\n1. Dimensionalidad:")
    for metodo, datos in resultados.items():
        print(f"{metodo.upper()}: {datos['matrix'].shape}")
    
    # Análisis de similaridad
    print("\n2. Similaridad entre Documentos:")
    for metodo, datos in resultados.items():
        sim_matrix = datos['similarity_matrix']
        print(f"\n{metodo.upper()}:")
        print(f"  Promedio de similaridad: {np.mean(sim_matrix):.4f}")
        print(f"  Máxima similaridad: {np.max(sim_matrix):.4f}")
        print(f"  Mínima similaridad: {np.min(sim_matrix):.4f}")
    
    # Análisis de vocabulario
    print("\n3. Características del Vocabulario:")
    for metodo, datos in resultados.items():
        vocab_stats = datos['vocab_stats']
        print(f"\n{metodo.upper()}:")
        print(f"  Términos totales: {vocab_stats['total_terms']}")
        print(f"  Longitud media de términos: {vocab_stats['avg_term_length']:.2f}")

# Ejecutar análisis comparativo
analisis_comparativo(resultados)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords

# Asegúrate de tener estas descargas
nltk.download('punkt')

class TextVectorizer:
    def __init__(self, documents, stop_words=None):
        """
        Inicialización del vectorizador de texto
        
        Args:
        - documents: Lista de documentos (texto plano)
        - stop_words: Conjunto de palabras vacías
        """
        self.documents = documents
        self.stop_words = stop_words or set(stopwords.words('spanish'))
    
    def bag_of_words(self, max_features=1000):
        """
        Vectorización usando Bag of Words
        
        Args:
        - max_features: Número máximo de características
        
        Returns:
        - Matriz de vectores BoW
        - Vocabulario
        """
        # Configurar vectorizador
        vectorizer = CountVectorizer(
            stop_words=list(self.stop_words),
            max_features=max_features
        )
        
        # Transformar documentos
        bow_matrix = vectorizer.fit_transform(self.documents)
        
        return {
            'matrix': bow_matrix.toarray(),
            'vocabulary': vectorizer.get_feature_names_out(),
            'vectorizer': vectorizer
        }
    
    def tf_idf(self, max_features=1000):
        """
        Vectorización usando TF-IDF
        
        Args:
        - max_features: Número máximo de características
        
        Returns:
        - Matriz de vectores TF-IDF
        - Vocabulario
        """
        # Configurar vectorizador TF-IDF
        vectorizer = TfidfVectorizer(
            stop_words=list(self.stop_words),
            max_features=max_features
        )
        
        # Transformar documentos
        tfidf_matrix = vectorizer.fit_transform(self.documents)
        
        return {
            'matrix': tfidf_matrix.toarray(),
            'vocabulary': vectorizer.get_feature_names_out(),
            'vectorizer': vectorizer
        }
    
    def word2vec(self, vector_size=100, window=5, min_count=1):
        """
        Vectorización usando Word2Vec
        
        Args:
        - vector_size: Dimensionalidad de los vectores
        - window: Contexto de palabras
        - min_count: Frecuencia mínima de palabras
        
        Returns:
        - Modelo Word2Vec
        - Matriz de vectores promedio de documentos
        """
        # Tokenizar documentos
        tokenized_docs = [
            nltk.word_tokenize(doc.lower()) 
            for doc in self.documents
        ]
        
        # Entrenar modelo Word2Vec
        model = Word2Vec(
            sentences=tokenized_docs, 
            vector_size=vector_size, 
            window=window, 
            min_count=min_count
        )
        
        # Calcular vector promedio por documento
        doc_vectors = []
        for doc in tokenized_docs:
            doc_vector = np.zeros(vector_size)
            count = 0
            for word in doc:
                if word in model.wv.key_to_index:
                    doc_vector += model.wv[word]
                    count += 1
            
            # Promedio de vectores
            if count > 0:
                doc_vector /= count
            
            doc_vectors.append(doc_vector)
        
        return {
            'model': model,
            'doc_vectors': np.array(doc_vectors)
        }

# Ejemplo de uso
documentos = [
    "El gato duerme en el sofá",
    "El perro juega en el jardín",
    "El gato y el perro son animales domésticos"
]

# Inicializar vectorizador
vectorizador = TextVectorizer(documentos)

# Bag of Words
bow_result = vectorizador.bag_of_words()
print("\n--- Bag of Words ---")
print("Matriz BoW:")
print(bow_result['matrix'])
print("\nVocabulario BoW:")
print(bow_result['vocabulary'])

# TF-IDF
tfidf_result = vectorizador.tf_idf()
print("\n--- TF-IDF ---")
print("Matriz TF-IDF:")
print(tfidf_result['matrix'])
print("\nVocabulario TF-IDF:")
print(tfidf_result['vocabulary'])

# Word2Vec
word2vec_result = vectorizador.word2vec()
print("\n--- Word2Vec ---")
print("Vectores de documentos:")
print(word2vec_result['doc_vectors'])